In [1]:
import sqlparse

from time import perf_counter

from django.test import RequestFactory
from django.db import connection, reset_queries

from cast.feeds import LatestEntriesFeed
from cast.models.repository import FeedRepository
from cast.models.repository import data_for_blog_cachable

In [2]:
def count_queries(func):
    reset_queries()
    renditions = list(func())
    return len(connection.queries)


def show_queries(queries):
    queries = sorted(queries, key=lambda x: x["time"], reverse=True)
    for num, query in enumerate(queries, 1):
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        duration = query["time"]
        print(f"{num}. query | duration: {duration}")
        print(formatted_sql)
        print("---------------")
        

def blocker(*_args):
    """Get a traceback when a query is executed."""
    raise Exception("No database access allowed here.")

In [3]:
# blog_slug = "ephes_blog"
blog_slug = "das_claas_und_nora_blog"
blog = Blog.objects.get(slug=blog_slug)

In [4]:
request_factory = RequestFactory()
request = request_factory.get(f"/blogs/{blog_slug}/feed/rss.xml")
request.htmx = False

In [5]:
%%time
reset_queries()
cachable_data = FeedRepository.data_for_feed_cachable(request=request, blog=blog)

CPU times: user 1.26 s, sys: 45.5 ms, total: 1.31 s
Wall time: 1.34 s


In [6]:
cachable_data["blog"]

{'id': 3,
 'pk': 3,
 'author': None,
 'uuid': UUID('b1901089-6ddc-4297-afe7-7c75187f6ed9'),
 'email': 'jochen-kinder@wersdoerfer.de',
 'comments_enabled': True,
 'noindex': True,
 'template_base_dir': 'bootstrap5',
 'description': '<p data-block-key="819mf">Was gerade so passiert</p>'}

In [7]:
%%time
repository = FeedRepository.create_from_cachable_data(data=cachable_data)

CPU times: user 693 ms, sys: 8.57 ms, total: 702 ms
Wall time: 701 ms


In [8]:
%%time
reset_queries()
# with connection.execute_wrapper(blocker):
feed_view = LatestEntriesFeed(repository=repository)
r = feed_view(request, slug=blog_slug)

get feed:  None <WSGIRequest: GET '/blogs/das_claas_und_nora_blog/feed/rss.xml'>
CPU times: user 2.49 s, sys: 198 ms, total: 2.69 s
Wall time: 2.7 s


In [9]:
len(r.content)

5467153

In [10]:
print(len(connection.queries))

1


In [17]:
# print(r.content.decode("utf8"))

In [18]:
show_queries(connection.queries)